In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

### Helper functions

In [ ]:
def save_result(path_name, file_name, result):
    """
    Save the result to a file.
    
    Args:
        path_name (str): Path to save the file
        file_name (str): Name of the file
        result (str): Content to save
    """
    file_path = os.path.join(path_name, f"{file_name}.md")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w') as file:
        file.write(result)
        
def setup_environment():
    """Set up the environment variables."""
    try:
        hf_key = os.environ["LLAMA3_KEY"]
    except KeyError:
        print("Please set the environment variable LLAMA3_KEY")
        hf_key = input("Enter your HuggingFace API key: ")
    return hf_key

def load_model(hf_key):
    """Load the LLM model and tokenizer."""
    quant_config = BitsAndBytesConfig(load_in_8bit=True)
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_key)
    model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_key, quantization_config=quant_config, device_map="auto", max_length=8192)
    config = AutoConfig.from_pretrained(model_name, token=hf_key)
    
    return model, tokenizer, config

def generate_response(model, tokenizer, messages, config):
    """Generate a response from the model."""
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to('cuda')

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    generated_ids = model.generate(
        model_inputs,
        do_sample=True,
        temperature=0.1,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        max_length=4500
    )

    print(f"Tokens used: {len(model_inputs[0])} out of {config.max_position_embeddings}")

    response = generated_ids[0][model_inputs.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

## 1. Setting up the model

This step includes downloading the LLM from Hugging Face and setting up the API key.

You can temporarily add your API key by using Python with os.environ["LLAMA3_KEY"]="YOUR_API_KEY"

WARNING: NEVER SHARE/PUSH YOUR API KEY.

In [ ]:
hf_key = setup_environment()
model, tokenizer, config = load_model(hf_key)

## 2. Set file directories

In [ ]:
# Set the CWD to the directory of your repository
os.chdir('/home/gunes/fair/responsible-ai-model-cards/Steps-v3-thesis')

### Individual case to analyze

In [ ]:
case_name = "norauto"

model_card_path = "files/cases/" + case_name + "/model-card.md" 
model_card = open(model_card_path, "r").read()

analysis_path = "files/cases/" + case_name + "/fairness-analysis.md"
analysis = open(analysis_path, "r").read()

log_path = "files/logs/" + case_name + ".txt"
logs = open(log_path, "r").read()

updated_model_card_path = "files/model-cards-final/"

## 3. Start Model Card Update

In [ ]:
messages = [{
  "role": "system",
  "content": """You are a specialist in model cards and responsible AI. Your task is to analyze the provided model card, measurement and mitigation methods, and intervention logs. Based on this analysis, you need to update the model card by adding a new section titled "Fairness Interventions".

The following files will be provided:
- **Model Card**: The current model card to be updated.
- **Measurement and Mitigation Methods**: This file includes the rationale for the selection of fairness measurement and mitigation methods.
- **Intervention Logs**: This file contains the results of the applied fairness interventions, including measurement results of fairness metrics and the overall performance of the mitigated model.

Update the model card by adding the "Fairness Interventions" section as outlined below. Focus on enhancing the "Fairness Interventions" section without altering other parts of the model card.
### Model Card Update Structure:
- **Existing Sections**: Print all current sections of the model card as they are.
- **Fairness Interventions**:
    - **Measurement and Mitigation Methods**:
      [Summarize the fairness measurement and mitigation methods used, including their justification.]
    - **New Performance**:
      [Provide the pre-intervention fairness, the post-intervention performance and the post-intervention fairness values.]
    - **Analysis**:
      [Analyze the results of the interventions, noting significant changes and their implications.]
### 
Only provide the final output without any additional comments or remarks."""
},
{
  "role": "user",
  "content": f"""Model card to be updated: {model_card}
Measurement and mitigation method decision: {analysis}
Interventions made and results: {logs}"""
},
]

In [ ]:
result = generate_response(model, tokenizer, messages, config)
print(result)

In [ ]:
model_card_name = case_name + "-model-card-updated"
save_result(updated_model_card_path, model_card_name, result)